# Exemplo com Vassoura e dataset PD Behavior
Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet` e aplicar a classe `Vassoura`.

In [ ]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs

# Carregar dataset de exemplo
df = pd.read_parquet('datasets/pd_behavior_example.parquet')
df.head()

In [ ]:
vsess = Vassoura(
    df,
    target_col='ever90m12',
    heuristics=['corr', 'vif'],
    thresholds={'corr': 0.9, 'vif': 10},
)
df_clean = vsess.run()
df_clean.head()

In [ ]:
# Gera relatório resumido
vsess.generate_report('example_report.html')